In [19]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import string
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import nltk
import warnings
warnings.filterwarnings('ignore')

C:\Users\Administrator\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Administrator\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [41]:
category_list = ['comp.sys.mac.hardware','rec.autos','misc.forsale','sci.space','talk.politics.guns']

In [47]:
new_groups = fetch_20newsgroups(categories=category_list,remove=('headers','footers','quotes'))

In [48]:
doc = new_groups.data
doc

["\n740 Turbo in UK was good for 124mph. Useful for blowing away VW Beetles, though I\nbelieve the Beetle corners better. \n\nI can say without any doubt that I have never been blown away by any Volvo, ever.\nI've been blocked into a few car parks though by shit-head Volvo owners who 'only thought they'd be a few minutes'. This does not happen with the owners of any other makes of car.",
 'I am looking for a working docking deck (deck that goes on back of\ncamera) for an old JVC GX-S700 Tube video camera.  Any format is\nacceptable.  Please send me a message if you even know anything about decks\nfor the GX-S700.  Also interested in any video equipment for sale,\nprofessional or consumer.  Thank you. \n\n----\nbbates@pro-freedom.van.wa.us   -==-   Pro-Freedom BBS - (206) 694-3276',
 "\n >In article <1993Apr19.020359.26996@sq.sq.com>, msb@sq.sq.com (Mark Brader) \n\nMB>                                                             So the\nMB> 1970 figure seems unlikely to actually be anyt

In [49]:
len(doc)

2896

In [50]:
type(new_groups)


sklearn.utils.Bunch

In [51]:
doc_df= pd.DataFrame(doc,columns={'Document':0})
doc_df.loc[0][0]

"\n740 Turbo in UK was good for 124mph. Useful for blowing away VW Beetles, though I\nbelieve the Beetle corners better. \n\nI can say without any doubt that I have never been blown away by any Volvo, ever.\nI've been blocked into a few car parks though by shit-head Volvo owners who 'only thought they'd be a few minutes'. This does not happen with the owners of any other makes of car."

In [57]:
doc_df['clean_text']=doc_df['Document'].str.lower().str.replace("[^a-z]"," ")
doc_df.head()

,Document,clean_text
0,\n740 Turbo in UK was good for 124mph. Useful ...,turbo in uk was good for mph useful f...
1,I am looking for a working docking deck (deck ...,i am looking for a working docking deck deck ...
2,\n >In article <1993Apr19.020359.26996@sq.sq.c...,in article apr sq sq co...
3,"A few posts back, somebody mentioned that the ...",a few posts back somebody mentioned that the ...
4,"\n\n\n\n\n\nIt still applies, except the astro...",it still applies except the astronomy t...


In [59]:
stop = set(stopwords.words('english'))

def sw(text):
    text = [word for word in text.split() if word not in stop] # spliting the words with space
    return " ".join(text) #

doc_df['Final'] = doc_df['clean_text'].apply(sw)
doc_df['Final'].head()

0    turbo uk good mph useful blowing away vw beetl...
1    looking working docking deck deck goes back ca...
2    article apr sq sq com msb sq sq com mark brade...
3    posts back somebody mentioned duo might crash ...
4    still applies except astronomy days long basel...
Name: Final, dtype: object

In [60]:
vec = TfidfVectorizer()
DTM = vec.fit_transform(doc_df['Final'])

In [61]:
lda = LatentDirichletAllocation(n_components= 5, max_iter=10,random_state =1234 )
lda_output = lda.fit_transform(DTM)
print(lda)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=1234,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)


In [62]:
#Documnet  -topic matrix

topic_name = ['Topic '+str(i) for i in range(lda.n_components)]
doc_name = [ 'Document '+str(i) for i in range(DTM.shape[0])]

In [64]:
df_document_topic = pd.DataFrame(np.round(lda_output,2),columns=topic_name,index=doc_name)
df_document_topic.head()

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4
Document 0,0.03,0.56,0.35,0.03,0.03
Document 1,0.03,0.87,0.03,0.03,0.03
Document 2,0.04,0.13,0.75,0.04,0.04
Document 3,0.03,0.87,0.03,0.03,0.03
Document 4,0.03,0.03,0.90,0.03,0.03


In [65]:
df_max_topic = np.argmax(df_document_topic.values,axis=1)

In [66]:
df_document_topic['df_max_topic'] = df_max_topic

In [67]:
df_document_topic.groupby('df_max_topic').size()

df_max_topic
0     103
1    1840
2     920
3      16
4      17
dtype: int64

In [68]:
df_document_topic.df_max_topic.value_counts()

1    1840
2     920
0     103
4      17
3      16
Name: df_max_topic, dtype: int64

In [70]:
#Creating the data frame between topic and terms

df_topic_keywords = pd.DataFrame(lda.components_,)


#Naming the columns and rows 

df_topic_keywords.columns= vec.get_feature_names()
df_topic_keywords.index = topic_name
df_topic_keywords

,aa,aaa,aaah,aacvkc,aad,aaef,aalm,aamazing,aamir,aamrl,...,zx,zxf,zxm,zxp,zxqi,zy,zyg,zyhszg,zyxel,zz
Topic 0,0.200032,0.200043,0.200021,0.200015,0.200026,0.200002,0.200015,0.200031,0.200056,0.200018,...,0.200090,0.200002,0.200002,0.200002,0.200002,0.200003,0.200002,0.200002,0.200015,0.200003
Topic 1,0.772186,0.324517,0.200010,0.252109,0.255192,0.200884,0.252109,0.377953,0.762685,0.200426,...,0.300704,0.200884,0.200884,0.200884,0.200884,0.201427,0.200884,0.200884,0.637961,0.201741
Topic 2,0.299467,0.519774,0.347700,0.200008,0.213638,0.200001,0.200008,0.200015,0.395503,0.555210,...,0.206206,0.200001,0.200001,0.200001,0.200001,0.200001,0.200001,0.200001,0.200008,0.200002
Topic 3,0.205322,0.200042,0.200020,0.200015,0.200025,0.210394,0.200015,0.200031,0.201500,0.200018,...,1.195922,0.210394,0.210394,0.210394,0.210394,0.221130,0.210394,0.210394,0.200015,0.232096
Topic 4,0.201340,0.200043,0.200021,0.200015,0.200026,0.200002,0.200015,0.200031,0.200056,0.200018,...,0.200090,0.200002,0.200002,0.200002,0.200002,0.200003,0.200002,0.200002,0.200016,0.200003


In [71]:
#finding the top 20 words for each topic

def show_topic(vec, lda, n_words):
    keywords = np.array(vec.get_feature_names())
    topic_keywords = []
    for topic_weights in lda.components_:
        topic_keywords_loc = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(topic_keywords_loc))
    return topic_keywords
topic_keywords = show_topic(vec= vec , lda =lda ,n_words = 20)

In [72]:

topic_keywords

[array(['feustel', 'kth', 'nada', 'hacker', 'tte', 'beamers', 'specint',
        'myi', 'deluxe', 'specfp', 'lojack', 'cain', 'egalon', 'preasure',
        'sherzer', 'methodology', 'hdfd', 'futon', 'chevrolet', 'sewing'],
       dtype='<U33'),
 array(['one', 'would', 'like', 'new', 'get', 'drive', 'car', 'thanks',
        'please', 'know', 'mac', 'sale', 'use', 'also', 'anyone', 'price',
        'apple', 'good', 'mail', 'used'], dtype='<U33'),
 array(['would', 'space', 'people', 'gun', 'one', 'think', 'could', 'like',
        'us', 'nasa', 'know', 'guns', 'get', 'even', 'much', 'first',
        'well', 'government', 'see', 'fbi'], dtype='<U33'),
 array(['test', 'comics', 'dj', 'col', 'ditto', 'ironic', 'zx',
        'chemistry', 'cdt', 'headline', 'beacons', 'nutek', 'fusi', 'rwf',
        'udel', 'brooks', 'danny', 'copyright', 'jj', 'fooled'],
       dtype='<U33'),
 array(['art', 'ites', 'feustel', 'chuvashia', 'inguiry', 'eridan',
        'elvis', 'wolverine', 'hulk', 'nasm', 'su',

In [73]:
Top_20 = pd.DataFrame(topic_keywords,index=topic_name,columns=np.arange(1,21))#columns=topic_name,
Top_20.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Topic 0,feustel,kth,nada,hacker,tte,beamers,specint,myi,deluxe,specfp,lojack,cain,egalon,preasure,sherzer,methodology,hdfd,futon,chevrolet,sewing
Topic 1,one,would,like,new,get,drive,car,thanks,please,know,mac,sale,use,also,anyone,price,apple,good,mail,used
Topic 2,would,space,people,gun,one,think,could,like,us,nasa,know,guns,get,even,much,first,well,government,see,fbi
Topic 3,test,comics,dj,col,ditto,ironic,zx,chemistry,cdt,headline,beacons,nutek,fusi,rwf,udel,brooks,danny,copyright,jj,fooled
Topic 4,art,ites,feustel,chuvashia,inguiry,eridan,elvis,wolverine,hulk,nasm,su,spider,er,disconcerting,eu,myi,ncsl,csmes,fahrvergnugen,renault
